<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/prueba_2_selene_json_gl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalación de dependencias
# Instalamos las librerías necesarias para cargar y usar el modelo
!pip install -q transformers accelerate

# Imports
# torch se usa para manejar tensores y ejecutar el modelo
# transformers permite cargar el modelo y el tokenizer
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# userdata se usa en Colab para leer variables secretas (como el token de Hugging Face)
from google.colab import userdata
import json  # para devolver la salida en formato JSON

In [4]:
# Token de Hugging Face
# Leemos el token guardado en los secrets de Colab
HF_TOKEN = userdata.get("HF_TOKEN")

# Comprobamos que el token existe antes de continuar
assert HF_TOKEN is not None, "No se encontró HF_TOKEN en los secrets de Colab"

In [5]:
# Modelo Selene-1-Mini
# Este modelo se usa como LLM-as-a-Judge binario para evaluar concordancia de número en gallego
MODEL_ID = "AtlaAI/Selene-1-Mini-Llama-3.1-8B"

print("🔄 Cargando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN
)

print("🔄 Cargando modelo (puede tardar unos minutos)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN,
    device_map="auto",            # usa GPU automáticamente si está disponible
    torch_dtype=torch.bfloat16    # formato eficiente para inferencia
)

# Ponemos el modelo en modo evaluación (muy importante)
model.eval()
print("✅ Modelo cargado correctamente")

🔄 Cargando tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

🔄 Cargando modelo (puede tardar unos minutos)...


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

✅ Modelo cargado correctamente


In [6]:
# Función LLM-as-a-Judge

# Esta función evalúa si una oración tiene o no
# errores de concordancia de número (singular/plural)
def selene_judge_concordancia(oracion: str) -> dict:
    # Prompt que explica claramente al modelo qué tiene que hacer
    prompt = f"""
Es un LLM-as-a-judge que vai avaliar un modelo de corrección gramatical en galego.

A túa tarefa é avaliar a saída do modelo de corrección gramatical (GEC)
e decidir se a corrección é ADECUADA con respecto á concordancia de número
(singular/plural entre determinante, substantivo, adxectivo ou verbo).

Devolve a resposta EXACTAMENTE co seguinte formato, sen texto adicional,
e empregando un formato JSON válido:

{{
  "output_modelo": "",
  "etiqueta": 0,
  "explicacion": ""
}}

Criterios:
- etiqueta = 1 → a corrección NON é adecuada
- etiqueta = 0 → a corrección é adecuada
- A explicación debe xustificar só a concordancia de número

NON DEBES CORRIXIR O TEXTO.
NON DEBES AVALIAR OUTROS TIPOS DE ERROS.

Agora avalía a seguinte saída dun modelo de GEC:

"{oracion}"
"""

    # Convertimos el prompt al formato de chat que espera Selene
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenizamos el texto y lo enviamos al mismo dispositivo que el modelo
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    # Generación sin muestreo para que sea determinista
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decodificamos solo la parte generada por el modelo
    resposta_texto = tokenizer.decode(
        output[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    ).strip()

    # Convertimos la salida del modelo (texto) en JSON
    # Si por cualquier motivo falla, devolvemos el texto crudo
    try:
        resposta_json = json.loads(resposta_texto)
    except json.JSONDecodeError:
        resposta_json = {
            "error": "La salida no es un JSON válido",
            "raw_output": resposta_texto
        }

    return resposta_json

Ejemplos con oraciones de prueba:

In [7]:
exemplos = [
    "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas.",
    "O grupo de estudantes que participaron no proxecto presentou os resultados finais onte pola tarde.",
    "As propostas que chegaron desde os concellos máis pequenos foi analizada polo equipo técnico.",
    "A maioría das persoas entrevistadas manifestaron a súa opinión durante a sesión pública."
]

# Ejecutamos el juez sobre cada ejemplo y mostramos el JSON resultante
for i, frase in enumerate(exemplos, 1):
    print(f"\n===== EXEMPLO {i} =====")
    resultado = selene_judge_concordancia(frase)
    print(json.dumps(resultado, ensure_ascii=False, indent=2))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



===== EXEMPLO 1 =====
{
  "output_modelo": "As decisións tomadas polo comité foron comunicadas aos responsábeis das distintas áreas.",
  "etiqueta": 1,
  "explicacion": "O substantivo'responsábeis' debe concordar en número co determinante 'aos responsables', que é plural."
}

===== EXEMPLO 2 =====
{
  "output_modelo": "O grupo de estudantes que participou no proxecto presentou os resultados finais onte pola tarde.",
  "etiqueta": 1,
  "explicacion": "O verbo 'presentou' debe ser no singular pois concorda con o suxeito 'grupo', que é singular."
}

===== EXEMPLO 3 =====
{
  "output_modelo": "As propostas que chegaron desde os concellos máis pequenos foi analisadas polo equipo técnico.",
  "etiqueta": 1,
  "explicacion": "O verbo 'fui' debe ser 'foron' para concordar co plural do substantivo 'propostas'."
}

===== EXEMPLO 4 =====
{
  "output_modelo": "A maioría das persoas entrevistadas manifestaron a súa opinión durante a sesión pública.",
  "etiqueta": 1,
  "explicacion": "O verbo'mani